# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [20]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to jitendragneogi@gmail.com and will expire on July 29, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\IBM_AD~1\AppData\Local\Temp\graphlab_server_1473179883.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.579090     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.609092     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
def print_coefficients(model):    
    # Get the degree of the polynomial
    deg = len(model.coefficients['value'])-1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print 'Learned polynomial for degree ' + str(deg) + ':'
    w.reverse()
    print numpy.poly1d(w)

In [14]:
print 'l1_penalty = %e' % 1e10
print 'number of nonzeros = %d' % (model_all.coefficients['value']).nnz()
print_coefficients(model_all)

l1_penalty = 1.000000e+10
number of nonzeros = 6
Learned polynomial for degree 17:
       14         13         5         4        3
20.02 x  + 842.1 x  + 350.1 x + 24.42 x + 8469 x + 2.749e+05


In [15]:
print all_features

['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [16]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [18]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    residuals = predictions - outcome
    # Then square and add them up
    residuals_square = residuals*residuals
    RSS = residuals_square.sum()
    return(RSS)  

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [23]:
lowest_rss = 0.0
lowest_l1_penalty = 0.0
for l1_penalty in np.logspace(1, 7, num=13):
    model1 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              verbose = False, validation_set = None,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    rss_validation = get_residual_sum_of_squares(model1, validation, validation['price'])
    print "Validation Error for l1 penalty value %f is %f"%(l1_penalty, rss_validation)   
    
    if (lowest_rss == 0.0 or lowest_rss > rss_validation):
        lowest_rss = rss_validation
        lowest_l1_penalty = l1_penalty
        
print ""
print "Lowest Validation Error for l1 penalty value %f is %f"%(lowest_l1_penalty, lowest_rss) 

Validation Error for l1 penalty value 10.000000 is 625766285142459.875000
Validation Error for l1 penalty value 31.622777 is 625766285362394.125000
Validation Error for l1 penalty value 100.000000 is 625766286057885.000000
Validation Error for l1 penalty value 316.227766 is 625766288257224.625000
Validation Error for l1 penalty value 1000.000000 is 625766295212186.750000
Validation Error for l1 penalty value 3162.277660 is 625766317206080.500000
Validation Error for l1 penalty value 10000.000000 is 625766386760658.125000
Validation Error for l1 penalty value 31622.776602 is 625766606749278.500000
Validation Error for l1 penalty value 100000.000000 is 625767302791634.125000
Validation Error for l1 penalty value 316227.766017 is 625769507643886.250000
Validation Error for l1 penalty value 1000000.000000 is 625776517727024.000000
Validation Error for l1 penalty value 3162277.660168 is 625799062845467.000000
Validation Error for l1 penalty value 10000000.000000 is 625883719085425.250000

L

In [28]:
model2 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              verbose = False, validation_set = None,
                                              l2_penalty=0., l1_penalty=lowest_l1_penalty)

   
rss_test = get_residual_sum_of_squares(model2, testing, testing['price'])

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [29]:
print "RSS on Test data for l1 penalty value %f is %f"%(lowest_l1_penalty, rss_test)   

RSS on Test data for l1 penalty value 10.000000 is 156983602381664.187500


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [30]:
print 'l1_penalty = %e' %lowest_l1_penalty
print 'number of nonzeros = %d' % (model2.coefficients['value']).nnz()
print_coefficients(model2)

l1_penalty = 1.000000e+01
number of nonzeros = 18
Learned polynomial for degree 17:
       17         16         15         14        13        12
56.07 x  + 9.434 x  + 122.4 x  + 43.29 x  + 6207 x  + 6609 x 
              11             10             9            8         7
 + 9.331e+04 x  + 6.019e+05 x  + 1.292e+04 x + 2.12e+04 x + 148.3 x
             6        5         4             3       2
 + 0.003484 x + 1125 x + 39.12 x + 2.541e+04 x + 937 x + 7937 x + 1.899e+04


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [36]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [33]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [57]:
for l1_penalty in l1_penalty_values:
    model3 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              verbose = False, validation_set = None,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    
    print 'l1_penalty = %e' %l1_penalty 
    print 'number of nonzeros = %d' % (model3.coefficients['value']).nnz() 
    current_non_zero = (model3.coefficients['value']).nnz() 
    if current_non_zero > max_nonzeros:
            l1_penalty_min = l1_penalty
            min_current_non_zero = current_non_zero
            
    if (current_non_zero < max_nonzeros) is True:
        l1_penalty_max = l1_penalty
        max_current_non_zero = current_non_zero
        break
            
print "" 
print l1_penalty_min
print 'number of nonzeros = %d' % min_current_non_zero

print "" 
print l1_penalty_max
print 'number of nonzeros = %d' % max_current_non_zero 

l1_penalty = 1.000000e+08
number of nonzeros = 18
l1_penalty = 1.274275e+08
number of nonzeros = 18
l1_penalty = 1.623777e+08
number of nonzeros = 18
l1_penalty = 2.069138e+08
number of nonzeros = 18
l1_penalty = 2.636651e+08
number of nonzeros = 17
l1_penalty = 3.359818e+08
number of nonzeros = 17
l1_penalty = 4.281332e+08
number of nonzeros = 17
l1_penalty = 5.455595e+08
number of nonzeros = 17
l1_penalty = 6.951928e+08
number of nonzeros = 17
l1_penalty = 8.858668e+08
number of nonzeros = 16
l1_penalty = 1.128838e+09
number of nonzeros = 15
l1_penalty = 1.438450e+09
number of nonzeros = 15
l1_penalty = 1.832981e+09
number of nonzeros = 13
l1_penalty = 2.335721e+09
number of nonzeros = 12
l1_penalty = 2.976351e+09
number of nonzeros = 10
l1_penalty = 3.792690e+09
number of nonzeros = 6

2976351441.63
number of nonzeros = 10

3792690190.73
number of nonzeros = 6


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [ ]:
l1_penalty_min = 
l1_penalty_max = 

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [58]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [61]:
lowest_rss = 0.0
for l1_penalty in l1_penalty_values:
    model4 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              verbose = False, validation_set = None,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    rss_validation = get_residual_sum_of_squares(model4, validation, validation['price'])
    print "Validation Error for l1 penalty value %f is %f"%(l1_penalty, rss_validation)   
    print 'number of nonzeros = %d' % (model4.coefficients['value']).nnz() 
    
    if (lowest_rss == 0.0 or lowest_rss > rss_validation) and ((model4.coefficients['value']).nnz()== max_nonzeros):
        lowest_rss = rss_validation
        lowest_l1_penalty = l1_penalty
        
print ""
print "Lowest Validation Error for l1 penalty value %f is %f"%(lowest_l1_penalty, lowest_rss) 

Validation Error for l1 penalty value 2976351441.631313 is 966925692362084.500000
number of nonzeros = 10
Validation Error for l1 penalty value 3019316638.952415 is 974019450084556.125000
number of nonzeros = 10
Validation Error for l1 penalty value 3062281836.273517 is 981188367942452.750000
number of nonzeros = 10
Validation Error for l1 penalty value 3105247033.594619 is 989328342459474.000000
number of nonzeros = 10
Validation Error for l1 penalty value 3148212230.915721 is 998783211265891.250000
number of nonzeros = 10
Validation Error for l1 penalty value 3191177428.236824 is 1008477167020094.000000
number of nonzeros = 10
Validation Error for l1 penalty value 3234142625.557926 is 1018298780553819.750000
number of nonzeros = 10
Validation Error for l1 penalty value 3277107822.879028 is 1028247992205977.250000
number of nonzeros = 10
Validation Error for l1 penalty value 3320073020.200130 is 1034616909232828.125000
number of nonzeros = 8
Validation Error for l1 penalty value 33630

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [62]:
model5 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              verbose = False, validation_set = None,
                                              l2_penalty=0., l1_penalty=lowest_l1_penalty)

In [63]:
print 'l1_penalty = %e' %lowest_l1_penalty
print 'number of nonzeros = %d' % (model5.coefficients['value']).nnz()
print_coefficients(model5)

l1_penalty = 3.448969e+09
number of nonzeros = 7
Learned polynomial for degree 17:
       14        13         5         4             3
30.01 x  + 2899 x  + 690.1 x + 32.41 x + 1.587e+04 x + 661.7 x + 2.223e+05
